# Adversarial example with multinomial logistic regression

In this example, we redo the [adversarial example](https://gurobi-optimization-ml2gurobi.readthedocs-hosted.com/en/latest/examples/adversarial_mnist.html) of the documentation but use a logistic regression model for the regression

We don't detail the optimization model here. Please refer to the example in the documentation.

## Import the necessary packages and load data

We import all the package we need for this example.
We fetch the MINST data set using sklearn's functionalities.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gurobipy as gp

In [ ]:
import sys
sys.path.append('../../src')

In [ ]:
from gurobi_ml import add_predictor_constr
from gurobi_ml.sklearn import logistic_regression

In [ ]:
# Get MNIST digit recognition data set
# Load data from https://www.openml.org/d/554
X, y = fetch_openml(
    "mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas"
)

# Flatten and scale the data

X = X.astype(np.float64) / 255.0
clf = LogisticRegression(C=50.0 / 500, penalty="l1", solver="saga", tol=0.1, random_state=4)
pipeline = make_pipeline(clf)
pipeline.fit(X, y)

In [ ]:
imageno = 15

In [ ]:
ex_prob = clf.predict_proba(X[imageno:imageno+1, :])
sorted_labels = np.argsort(ex_prob)[0]
right_label = sorted_labels[-1]
wrong_label = sorted_labels[-2]

In [ ]:
image = X[imageno, :]
plt.imshow(image.reshape((28, 28)), cmap="gray")
label = clf.predict(image.reshape(1,-1))
print(f"Solution is classified as {label}")

In [ ]:
wrong_label

In [ ]:
m = gp.Model()
delta = 10

x = m.addMVar(image.shape, lb=0.0, ub=1.0, name="x")
y = m.addMVar(ex_prob.shape, lb=-float('inf'), name="y")

abs_diff = m.addMVar(image.shape, lb=0, ub=1, name="abs_diff")

m.setObjective(y[0, wrong_label] - y[0, right_label], gp.GRB.MAXIMIZE)


# Bound on the distance to example in norm-1
m.addConstr(abs_diff >= x - image)
m.addConstr(abs_diff >= -x + image)
m.addConstr(abs_diff.sum() <= delta)

pred_constr = add_predictor_constr(m, pipeline.steps[-1][1], x, y, epsilon=0, predict_function='predict')

pred_constr.print_stats()

In [ ]:
m.optimize()

In [ ]:
pred_constr.linear_predictor

In [ ]:
# Error in approximation
pred_constr.get_error().max()

In [ ]:
clf.decision_function()

In [ ]:
plt.imshow(x.X.reshape((28, 28)), cmap="gray")
label = pipeline.predict(x.X.reshape(1,-1))
print(f"Solution is classified as {label}")

copyright © 2023 Gurobi Optimization, LLC